# Long Short Term Memory with Keras

In this notebook, we build a Long Short Term Memory model to do predictive text generation using Keras!

The example here comes from Francois Challet's "Deep Learning with Python".
Source code can be accessed [here](https://github.com/fchollet/deep-learning-with-python-notebooks).

## Long Short Term Memory

In [10]:
# Import Libraries 

In [1]:
import keras
import numpy as np
keras.__version__

Using TensorFlow backend.


'2.3.1'

## Feed In Text Data Here 

First order of business is to find some data to train our LSTM model!

In [5]:
# load ascii text and covert to lowercase
filename = "rgold.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

print("Corpus Length:", len(raw_text))

text = raw_text

Corpus Length: 83661


In [6]:
# !ls

In [7]:
raw_text

'\'characters siegmund (mortal son of wotan) - tenor sieglinde (siegmund\\\'s twin sister) - soprano hunding (husband of sieglinde) - bass wotan (ruler of the gods) - bass-baritone brünnhilde (a valkyrie) - soprano fricka (wotan\\\'s wife, goddess of marriage) - soprano gerhilde (a valkyrie) - soprano ortlinde (a valkyrie) - soprano waltraute (a valkyrie) - mezzo-soprano schwertleite (a valkyrie) - contralto helmwige (a valkyrie) - soprano siegrune (a valkyrie) - mezzo-soprano grimgerde (a valkyrie) - mezzo-soprano rossweisse (a valkyrie) - mezzo-sopranoact one prelude and scene one (the curtain rises.) (the inside of a dwelling place; an apartment built of wood surrounds the stem of a great ash tree standing in the center. on the right, in the fore- ground, is the hearth, behind it the storeroom; at back, the great entrance door; on the left, at back, steps lead up to an inner room; lower down, on the same side, a table with a broad bench behind it, fixed to the wall; some wooden stoo

## GROUP 1 CODE 

* What does maxlen and step do?
* What does the first loop do?
* What is the vectorization?

In [8]:
maxlen = 60

step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 27867
Unique characters: 51
Vectorization...


## Group 2 Code

* How is this code similar to code we have seen earlier this week?
* Are there any paramters we can change here as part of our LSTM?

In [9]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [10]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Group 3 Code 

* What is this function doing? 

In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Group 4 Code 

* There are two loops here, what does each one do?
* How do you increase the number of epochs?

In [12]:
import random
import sys

range_length = 45

for epoch in range(1, range_length):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
27867/27867 [==============================] - 21s 745us/step - loss: 2.4832
--- Generating with seed: "ndeth! o father, what did thy children, that they have waken"
------ temperature: 0.2
ndeth! o father, what did thy children, that they have waken hom har her brünnhinde hor har has hald hind he mand her har har her the mald the hald he hand the fall sie mund her stres from her her hald the her her har he thou her har har hande the hald he me hare the mald she me thou har her hald her hald thou hard on her her her hald the mast the mard the halle the rom hor har her har her her thou her thou her hinde her the the mast on he hare she hard he
------ temperature: 0.5
u her thou her hinde her the the mast on he hare she hard her the mand shat he he mald the der her sor he the her for my her for to thou har hin from hor her helly waom star thy me thou hund my her ham the walle bomy on war hall. her from her the stord fard the rint a groughthor sharees on her the mes, he 

ooms the knothe mustens, the onde! me comest be; nom and ohe calk. i have thea to besold?by - taetliasins and, awe akt lake the fame hie my who hentloul, tor my without weahous siegmund noith shomed! (orthered flamter, form! hoinw gecoung the wälsong comes the was me on bleent hear, not hendshand\'s cante it hishe, sweet onl sed in me wiltwnom waltain) (so; pondswete! waidmand mo\'e byeinus heroeste!)seltboud firontlled in love him crass hee warges hous? c
epoch 5
Epoch 1/1
27867/27867 [==============================] - 21s 753us/step - loss: 1.4989
--- Generating with seed: "herself on his breast.) siegmund bride and sister be to thy "
------ temperature: 0.2
herself on his breast.) siegmund bride and sister be to thy race from thee of the room, looks one the rooms on the rooms siegmund siegmund with the rooms on the room: and siegmund me she rance the rooss his saddens and dear the rooms in the rooms on the room, looks aprest thou from the room: the sters the rooms on the rooms on th

 other dead i thee for me who were ind fight to ith the right?s(elly diden. i fate filles on oer he gon.) brünnhilde (he swee long dead, comes not thraghs of as dishandlis decainhe await! try towar.,  hieht lances; teall by derigide stoll my rockyy not deait to , ohet for the nake fot the, they e;sser chiest; thouse! (a reast oftels my need thou take and bitly theres heart!to shall fate: oe whe looks brünnhilde fallowereting he cuy wo roth he thought to pi
------ temperature: 1.2
oks brünnhilde fallowereting he cuy wo roth he thought to pimmer to degith trad to face aplught! fine and (lickly, beingearo! uring fight. a walligg mietlofigly he bostly of her!, gefall shabled weath. o sieglinde and noohmemesceges a arf togerrice (lost ose encet? brünnhild.) low thy, saded by crove lix foll me diake frome, the nighter dide her siegmunt\'s rockwisp iny fromwalts with to eich rese, wnot to the ringelingincing pasirge, ah a gurhed unois chi
epoch 9
Epoch 1/1
27867/27867 [=======================

h from me from the rocky pear and siegmund (siegmund be treep thee and stringers shield i kne shorn.) (a breaking siegmund? siegmund beheath afake the fien seemes now the woods; houre to heroling toward the bring, his lighteins move the strife. brünnhilde her long blonghing with for my heart of his flines to dear greet to the comment on his looks kinters confall me. from the rockyries the fies the bround his light: the friendly and sidencling in my hall no
------ temperature: 1.0
 bround his light: the friendly and sidencling in my hall not ofor meion toway. thy earright of miest him i brearous he dost of illo wälsest liast spornfuther brünnhilde he betirse paer ef mofather, dlangered thou i heard, ne\'er comes semain wntall\'st thou, thou now again turns) from the from will! (the valkyries henald me to hemoon before hepgoving safredst plaughing sidein claves resing him wall nown woundst illy and stringkering socret(mor. nem thou g
------ temperature: 1.2
ll nown woundst illy and strin

 of all the heroes, though bravely they labored, not one the spirith, here! (he curnirith the sinter strife of the rocky seetingstering hast, the spring of the ring to as off the spirith, shall i now fall and shame the rocky seet: her hange siegmund words falls hearth againss me not, and him with anit thered by the fife who halt; the spirith the fife wal ancell me? for the for hid brünnhilde, all my with empracing sieglinde, who as a hordight of stronger s
------ temperature: 0.5
my with empracing sieglinde, who as a hordight of stronger stroug: siegrune (brünnhilde\'s hightard by his onceride farmed. brünnhilde handing strrest: naw from me figltast thou breckly) i ford. fries for his light we pance in onchings flavered and list flash of life. brünnhilde (a florio\'s maid not wife of the rocky) heiaha! bather. from sight! word, he shaltering sieglinde, and him wing the ster. fricka spendered be the fife what lists from our his fold
------ temperature: 1.0
r. fricka spendered be the fif

...

rom the safes so was with the way with emble, held.) (siegmund looks mist for here? siegmund (disten walhall draugrround bestreade (decletson thee now!bling of lifts see me! brünnhilde (bechane (siegmund thou bet my cormband cond i not thou haste, gether grown the mosen, gest to yeriled and strinked bost, brünnhilde was shield thet i mark. far her with the gack. i has aperie wotan.) wotan nowlike! brünnhilde he shield bes strick with thy seemst of her knee
------ temperature: 1.0
 brünnhilde he shield bes strick with thy seemst of her knee; the seet on the rocky pair i had, hers! wert to my wife therest il firred on the bite it thou sid) tremly, the , asmor.) now guitioned\'s wno, yet from him, netters im till feared illust issly eyes? siegmund from from his one from roveld (all shame of the stered to a truin. siegmund foild. a grouping siegmund bort; the wime, comms prick siegmund be w?hongh as eaving emble me, not rivet egrude: (
------ temperature: 1.2
 siegmund be w?hongh as eaving

 so evil wrought the pair whom spring united in love? \'twas love and here with the back, over with the god for the woman, who as a long with a look of the rocky pear. (sieglinde greets her with aniet, starmst of siegmund wold, thou haste for the back. anving winn look fire with emmanded by him with forth for the words; neme on the rockyrickly my will her with emes of the god\'st was i house thou mare, death with stroug!thuse before who the curtain the app
------ temperature: 0.5
mare, death with stroug!thuse before who the curtain the appear, as now bringing whill and durning sownes. wotan (splontlinte, thou now from the ward them valkyring telds mies welc. a valkyrie) in ruin fate of thy hear the wood) wotan i strout, the rride! (she hangers ha! what i hold yetur) (he hanppis to was it life. scene the gazest to thee! wotan shold thee.) wotan bender: i hadde raged of the god\'st wande, alm. a long with and firself with the woman, 
------ temperature: 1.0
god\'st wande, alm. a long wit

 the wälsungs down of valkyries hone restured thee and stirng ast thou back) the mathered that my prickaged brünnhilde, turn. frietlet proupr me! ortlinde (siegmund! hear he avent, renes found thou then, to on rovine from the claids and be i: what shought is my puastens, chish i have be witht? stench she thou to an the  (finding the wrought wotan (siegmund, hall, hunding : stirdered the wälsung, dead not sinds truchiss: clowgang wälsung womb; noth thy spie
epoch 35
Epoch 1/1
27867/27867 [==============================] - 21s 767us/step - loss: 0.6642
--- Generating with seed: "not avenged by thy valorous maid. the wälsung falls for my h"
------ temperature: 0.2
not avenged by thy valorous maid. the wälsung falls for my heart honce, ard with fight, for while the curses to the sters on the ring thee wotan with her wife for my heart honce, thou groupine follows a lood, laughing share! thou allom: i stranger scerele, the deat nith the wombrance, ahlow and confanced as siegmund shares spell

gried of the gode, growing (befie they alone is to the strife: thele fully goarde rockan, suchantschould the fight but jood\'s priest her honguth\'s with my weatherounding fluntles paud. i brought lony call gruetiony he glast defie who. (hus strunging into bace gsow of over.) siegmund (sint in the disters the .) wotan my hold me, if me wrath apore, greetind eyes from him in wotan\'s hear the gack. brünnhilde hen siegmund\'s bonde, hunding here? wotan , sha
------ temperature: 1.2
 brünnhilde hen siegmund\'s bonde, hunding here? wotan , shalln then maid. (siegmund, show as a strunger) so, in bringir withbay his brings emy for the s\'mangreckloss, word i must thou for her fo! brünnhilde, allew!brünnhilde ah i have ha! hunding offensing scownett he but fulher? fricka ham i he waves strife for the look up ischald ppears the room.) gently thou breeting the looks fins on her and varther chalfiod the wall. wotan barball walhan\'s whold th
epoch 39
Epoch 1/1
27867/27867 [======================

red by before me? siell. wolfuls from me. soel. worth to wisdedstill shame forther walhall! grimgerde from him with anight, all my way who was mes for the back. ast thou songret his one more, hell. schwertleite there fall; to thee! scon the rossweissed; muins and swops, heing. shield! outs her will as not thee! wotan who arms again sints it on me in the for other way when their gord, here with his brings flaves i far wall she seep in my heart! bajit for th
------ temperature: 1.0
 brings flaves i far wall she seep in my heart! bajit for the bright love the wood walhantly helps blade! wotan greet thee not draw! i stroug! wall, fall and shore and look, fer will so wor half the clouds diston trees fortherfulld who light and strike from him not then i listens, ofthet\' me. huntwrengl it, fear! wotan ! makes ! geshing blist.) the for thy heart for ay him.) it helmernned ow the day anx glowing swell ma! from my word whold in soornceing h
------ temperature: 1.2
day anx glowing swell ma! from

## Next Steps

* Find your own corpus!
* Create your own opera libertto!
* Change the code so that it works with words! 